# Sample Code for the Adaptive Wind Driven Optimization

A simple sample Python code for the Adaptive Wind Driven Optimization is presented below. The coefficients of the WDO are optimized by the CMEAS algorithm. No additional cost function calls are needed for CMAES runs.

#### Contact Information:
- Zikri Bayraktar, Ph.D. 
- Email: thewdoalgorithm@gmail.com

### DISCLAIMER: This sample code is provided as is and only for educational purposes.  Please also read the referred papers to better understand the variables as well as the inner-workings of the algorithms. Use it at your own risk! There is no guarantee that the code is bug free.
<br>



WDO REFERENCE PAPERS:
Please refer to the following <b>TWO</b> articles in your publications if you use this code:

1.) Z. Bayraktar, and M. Komurcu, "Adaptive Wind Driven Optimization," 
Proceedings of the 9th EAI International Conference on Bio-inspired Information and 
Communications Technologies (Formerly BIONETICS) on 9th EAI International Conference 
on Bio-inspired Information and Communications Technologies (Formerly BIONETICS), 
New York City, NY, Dec. 3-5, 2015.
http://dl.acm.org/citation.cfm?id=2954811&CFID=806983354&CFTOKEN=67304345

2.) Z. Bayraktar, M. Komurcu, J. A. Bossard and D. H. Werner, "The Wind 
Driven Optimization Technique and its Application in Electromagnetics," 
IEEE Transactions on Antennas and Propagation, Volume 61, Issue 5, 
pages 2745 - 2757, May 2013.
http://ieeexplore.ieee.org/xpl/articleDetails.jsp?arnumber=6407788&newsearch=true&queryText=wind%20driven%20optimization



In [1]:
# import some modules:
from ctypes import *
from numpy.ctypeslib import ndpointer
import numpy as np
import sys
import platform

import math
import random
import scipy as sp 
import numpy.matlib
#from numpy import linalg as LA
from numpy import linalg as LA
from IPython.core.debugger import set_trace

### Define cost function here:

In [2]:
def CostFunct(x):
    return np.sum( (x-5) * (x-5) )

### Define the modified version of the CMAES, which optimizes WDO inherent parameters:

In [3]:
def cmaes(counteval, rec, npop, pres, dim): 
    # Refer to purecmaes.m -- https://www.lri.fr/~hansen/purecmaes.m
    # 
    # counteval -- Iteration counter from WDO.
    # rec -- Record of prior values used in CMAES.
    # npop -- number of population members from WDO, each member gets their own set of coefficients determined by the CMAES.
    # pres -- pressure(cost function) computed by WDO for the set of coefficients that CMEAS picked last iteration
    # dim -- number of dimensions of CMAES optimization
    
    if counteval==1:   #Initialization step
        # define 'records' dictionary to keep track the CMAES values over iterations. 
        #print('Init Stage')
        rec['N'] = dim
        rec['xmean'] = np.random.rand(dim,1)
        rec['sigma'] = 0.5
        rec['lambda'] = npop
        rec['mu'] = npop/2
        rec['weights'] = np.log((npop/2)+1/2) - np.log(np.arange(1,np.floor(npop/2)+1))
        rec['mu'] = np.floor(rec['mu']).astype(int)
        rec['weights'] = rec['weights']/np.sum(rec['weights'])
        rec['mueff'] = np.square(np.sum(rec['weights'])) / np.sum( rec['weights'] * rec['weights'])
        rec['cc'] = (4+rec['mueff']/rec['N']) / (rec['N']+4 +2*rec['mueff']/rec['N'])
        rec['cs'] = (rec['mueff']+2) / (rec['N'] + rec['mueff']+5)
        rec['c1'] = 2 / ((np.square(rec['N']+1.3)) + rec['mueff'])
        rec['cmu'] = min(1-rec['c1'], 2*(rec['mueff']-2+1/rec['mueff'])/(np.square(rec['N']+2)+rec['mueff']))
        rec['damps'] = 1 + 2*max(0, np.sqrt((rec['mueff'] -1)/(rec['N']+1))-1) + rec['cs']
        rec['pc'] = np.zeros(dim)
        rec['ps'] = np.zeros(dim)
        rec['B'] = np.eye(dim,dim)
        rec['D'] = np.ones(dim)
        rec['C'] = np.matmul(  np.matmul( rec['B'], np.diag(np.power(rec['D'],2)) ) , rec['B'].T  )
        rec['invsqrtC'] = np.matmul( np.matmul(rec['B'], np.diag(np.power(rec['D'],-1))) , rec['B']) 
        rec['eigeneval'] = 0
        rec['chiN'] = np.power(rec['N'],0.5)* (1-1/(4+rec['N'])+1/(21*np.square(rec['N'])) )
        
        
    #get fitness from WDO pressure
    rec['arfitness'] = pres
    # sort fitness and compute weighted mean into xmean
    arindex = np.argsort(pres)
    rec['arindex'] = arindex
    rec['arfitness'] = np.sort(pres)
    rec['xold'] = rec['xmean']
    mu = rec['mu']
    ridx = arindex[0:mu.astype(int)]
    recarx = np.array(rec['arx'])
    rec['xmean'] = np.matmul(  recarx[:,ridx], rec['weights'] ).reshape(dim,1)            
    
    rec['ps'] = (1-rec['cs']) * rec['ps'] + np.sqrt(rec['cs']*(2-rec['cs'])*rec['mueff'])  * np.matmul(rec['invsqrtC'] , (rec['xmean']-rec['xold'])).T / rec['sigma']
    rec['hsig'] = int( np.sum(rec['ps']*rec['ps']) / (1-np.power((1-rec['cs']),(2*counteval/rec['lambda']))) / rec['N']  <  2+(4/(rec['N']+1))   )
    rec['pc'] = (1-rec['cc']) * rec['pc'] + rec['hsig'] * np.sqrt(rec['cc']*(2-rec['cc'])*rec['mueff']) * (rec['xmean']-rec['xold']).T / rec['sigma']       
    rec['artmp'] = (1/rec['sigma']) * (recarx[:,ridx]) -np.matlib.repmat(rec['xold'],1,rec['mu'])

    rec['C'] = (1-rec['c1']-rec['cmu']) * rec['C'] \
                    + rec['c1'] * (rec['pc'] * rec['pc'].T + (1-rec['hsig']) * rec['cc']*(2-rec['cc']) * rec['C']) \
                    + rec['cmu'] * np.matmul( np.matmul(rec['artmp'] , np.diag(rec['weights'])) , rec['artmp'].T)
    
    rec['sigma'] = rec['sigma']*np.exp( (rec['cs']/rec['damps'])*(LA.norm(rec['ps'])/rec['chiN']-1) )
    
    if (counteval-rec['eigeneval']) >  (rec['lambda'] / (rec['c1']+rec['cmu'])/rec['N']/10):
        rec['eigeneval'] = counteval
        rec['C'] = np.triu(rec['C']) + np.triu(rec['C'],1).T
        rec['D'], rec['B'] = LA.eigh(rec['C'])
        rec['D'] = np.sqrt(rec['D'])
        rec['invsqrtC'] = np.matmul( np.matmul(rec['B'], np.diag( rec['D']**(-1)) )  , rec['B'].T)
        
        
    for k in range(1,rec['lambda']):
        recarx[:,k] = rec['xmean'].T + (rec['sigma']* np.matmul(rec['B'], ((rec['D']*( np.random.standard_normal(size=(rec['N'],1))).T)).reshape(dim,1) )).T
    rec['arx'] = recarx
    
    return rec

### Define WDO parameters and run initial population

In [4]:
popsize = 100    # population size
npar = 10        # dimension of the problem
maxit = 3000     # max number of iterations
maxV = 0.3      # max allowed speed
dimMin = -10      # lower dimension boundary
dimMax = 10      # upper dimension boundary

# randomly initialize the CMAES population
rec = {'arx': np.random.rand(4,popsize) }
alp = rec['arx'][0,:]
g = rec['arx'][1,:]
c = rec['arx'][2,:]
RT = rec['arx'][3,:]
#----------------------------------------------------------

# Initialize WDO population position, and velocity:
# Randomize population in the range of [-1,1]:
pos = 2 * (np.random.rand(popsize, npar) - 0.5)
vel = maxV * 2 * (np.random.rand(popsize, npar) - 0.5)
#----------------------------------------------------------

# Evaluate initial population: (Sphere Function)
pres = np.zeros((popsize))  # initialize the pres vector to zeros.
a=pres.shape # size of the pres

for i in range(0, popsize): #pyhon index starts from 0 !!!
    x = ((dimMax-dimMin) * (pos[i,:]+1)/2) + dimMin
    pres[i] = CostFunct(x)

In [5]:
#----------------------------------------------------------
# Finding best air parcel in the initial population :

globalpres = pres.min()         # minimum pressure
minIndx = np.where(pres == pres.min())  # index of minimum pressure
globalpos = pos[minIndx,:]       # position vector for the minimum

minpres = np.zeros(maxit)
keepglob = np.zeros(maxit)

indx = np.argsort(pres)      # index of sorted
pos = pos[indx,:]
minpres[0] = globalpres       # save the minimum pressure
keepglob[0] = globalpres;   
#-----------------------------------------------------------------

velot = np.zeros((popsize, npar))
keepglob = np.ones(maxit)

In [6]:
# Start iterations:
itr = 1 #iteration counter
for ij in range(1,maxit):
    #update velocity
    for i in range(popsize):
        a = np.random.permutation(range(0,npar)) #random perm    
        velot[i,:] = 1*vel[i,a]
        vel[i,:] =  (1-alp[i]) * vel[i,:] - (g[i]*pos[i,:]) + \
                    abs((1/(i+1))-1) *((globalpos-pos[i,:]*RT[i])) + \
                    (c[i]*velot[i,:]/(i+1))
        #python index starts from zero, watch out for division by zero error!

    #check velocity
    vel=vel.clip(-maxV,maxV)
    #update air parcel position
    pos=pos+vel
    pos=pos.clip(-1,1)
        
    #evaluate the new position
    for i in range(0, popsize): #pyhon index starts from 0 !!!
        x = ((dimMax-dimMin) * (pos[i,:]+1)/2) + dimMin
        pres[i] = CostFunct(x)

    # Finding best air parcel in the initial population :
    mpres = pres.min()         # minimum pressure
    mIndx = np.where(pres == pres.min())  # index of minimum pressure
    gpos = pos[mIndx,:]       # position vector for the minimum

    indx = np.argsort(pres)     # index of sorted
    pos = pos[indx,:]           #sort position
    vel = vel[indx,:]           #sort velocity
    
    #update inherent parameters through cmaes:
    x,y = rec['arx'].shape
    rec = cmaes(ij,rec,popsize,pres,x)
    alp = rec['arx'][0,:]
    g = rec['arx'][1,:]
    c = rec['arx'][2,:]
    RT = rec['arx'][3,:]
    
    if mpres < globalpres:  #if lower pressure found, update the global min
        globalpres = mpres
        globalpos = gpos
        print('iteration ',ij, 'pressure ' ,mpres)

    keepglob[ij] = globalpres
    
    
    if mpres <= 1e-7:
        break    #finish

iteration  1 pressure  112.002930264
iteration  2 pressure  77.6782421718
iteration  3 pressure  34.6553819199
iteration  4 pressure  15.8199798776
iteration  5 pressure  14.5725239689
iteration  6 pressure  5.5210792114
iteration  9 pressure  4.9951574577
iteration  10 pressure  2.80389357607
iteration  11 pressure  2.69758333218
iteration  12 pressure  1.77917957703
iteration  13 pressure  1.68219559815
iteration  14 pressure  1.20588366097
iteration  15 pressure  0.625919179967
iteration  17 pressure  0.37412616168
iteration  18 pressure  0.258413895694
iteration  19 pressure  0.150420951101
iteration  20 pressure  0.114540927781
iteration  28 pressure  0.097629896091
iteration  83 pressure  0.0


In [7]:
# End of file